In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/Kannada-MNIST/sample_submission.csv
/kaggle/input/Kannada-MNIST/train.csv
/kaggle/input/Kannada-MNIST/Dig-MNIST.csv
/kaggle/input/Kannada-MNIST/test.csv


In [2]:
%%time
train_data = pd.read_csv('/kaggle/input/Kannada-MNIST/train.csv')
test_data = pd.read_csv('/kaggle/input/Kannada-MNIST/test.csv')

CPU times: user 3.13 s, sys: 632 ms, total: 3.76 s
Wall time: 3.85 s


In [3]:
train_data.groupby(by='label').size()

label
0    6000
1    6000
2    6000
3    6000
4    6000
5    6000
6    6000
7    6000
8    6000
9    6000
dtype: int64

In [4]:
IMG_SIZE = 28

In [5]:
from keras.utils import to_categorical
img_train = train_data.drop(["label"], axis=1).values.reshape(-1, IMG_SIZE, IMG_SIZE, 1).astype('float32')
img_label = to_categorical(train_data["label"])

img_test = test_data.drop(["id"], axis=1).values.reshape(-1, IMG_SIZE, IMG_SIZE, 1).astype('float32')

print("img_train.shape = ", img_train.shape)
print("img_label.shape = ", img_label.shape)
print("img_test.shape = ", img_test.shape)

Using TensorFlow backend.


img_train.shape =  (60000, 28, 28, 1)
img_label.shape =  (60000, 10)
img_test.shape =  (5000, 28, 28, 1)


In [6]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(img_train, img_label, test_size=0.15)
print("x_train.shape = ", x_train.shape)
print("y_train.shape = ", y_train.shape)
print("x_test.shape = ", x_test.shape)
print("y_test.shape = ", y_test.shape)

x_train.shape =  (51000, 28, 28, 1)
y_train.shape =  (51000, 10)
x_test.shape =  (9000, 28, 28, 1)
y_test.shape =  (9000, 10)


In [7]:
import keras
from keras.datasets import mnist
from keras.layers import Input, Dense, Dropout, Flatten, add
from keras.layers import Conv2D, Activation, MaxPooling2D, AveragePooling2D
from keras import backend as K
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
from keras.models import Model
from keras.utils import plot_model
from keras.preprocessing. image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Dense, PReLU, Dropout
from keras.models import Model
from keras.callbacks import LearningRateScheduler, ModelCheckpoint, TensorBoard, EarlyStopping, ReduceLROnPlateau
from keras.optimizers import SGD, Adam

In [8]:
def res_block(x, channels, i):
    if i == 1:
        strides = (1, 1)
        x_add = x
    else:
        strides = (2, 2)
        x_add = Conv2D(channels,
                       kernel_size=(3, 3),
                       activation='relu',
                       padding='same',
                       strides=strides)(x)
 
    x = Conv2D(channels,
               kernel_size=(3, 3),
               activation='relu',
               padding='same')(x)
    x = Conv2D(channels,
               kernel_size=(3, 3),
               padding='same',
               strides=strides)(x)
    x = add([x, x_add])
    Activation(K.relu)(x)
    return x

In [9]:
def build_model(input_shape=(28, 28, 1), classes = 10):
    input_layer = Input(shape=input_shape)
    x = Conv2D(16, kernel_size=(7, 7), activation='relu', input_shape=input_shape, padding='same')(input_layer)
    for i in range(2):
        x = res_block(x, 16, i)
 
    for i in range(2):
        x = res_block(x, 32, i)
 
    x = AveragePooling2D(pool_size=(7, 7))(x)
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(classes, activation='softmax')(x)

    model = Model(inputs=input_layer, outputs=x)
    return model

In [10]:
model = build_model(input_shape=(28, 28, 1), classes = 10)

In [11]:
train_datagen = ImageDataGenerator(
    rotation_range=9, 
    zoom_range=0.25, 
    width_shift_range=0.25, 
    height_shift_range=0.25,
    rescale=1./255
)
train_datagen.fit(x_train)
test_datagen = ImageDataGenerator(rescale=1./255)
adam = Adam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=5, verbose=1, factor=0.5, min_lr=0.00001)
checkpoint = ModelCheckpoint("bestmodel.model", monitor='val_acc', verbose=1, save_best_only=True)
earlyStopping = EarlyStopping(monitor='val_acc', patience=15, verbose=1, mode='min')

In [15]:
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [13]:
epochs = 50
batch_size = 64

In [16]:
history = model.fit_generator(
    train_datagen.flow(x_train, y_train, batch_size=batch_size),
    steps_per_epoch=x_train.shape[0] // batch_size,
    epochs=epochs,
    validation_data=test_datagen.flow(x_test, y_test),
    validation_steps=x_test.shape[0] // batch_size,
    callbacks=[checkpoint, learning_rate_reduction])

Epoch 1/50
796/796 [==============================] - 17s 21ms/step - loss: 0.1219 - acc: 0.9663 - val_loss: 0.0342 - val_acc: 0.9911
Epoch 2/50
796/796 [==============================] - 16s 20ms/step - loss: 0.0695 - acc: 0.9789 - val_loss: 0.0309 - val_acc: 0.9940
Epoch 3/50
796/796 [==============================] - 16s 20ms/step - loss: 0.0676 - acc: 0.9801 - val_loss: 0.0308 - val_acc: 0.9928
Epoch 4/50
796/796 [==============================] - 16s 20ms/step - loss: 0.0653 - acc: 0.9814 - val_loss: 0.0189 - val_acc: 0.9935
Epoch 5/50
796/796 [==============================] - 15s 19ms/step - loss: 0.0639 - acc: 0.9807 - val_loss: 0.0214 - val_acc: 0.9944
Epoch 6/50
796/796 [==============================] - 15s 19ms/step - loss: 0.0636 - acc: 0.9810 - val_loss: 0.0451 - val_acc: 0.9873
Epoch 7/50
796/796 [==============================] - 16s 20ms/step - loss: 0.0660 - acc: 0.9808 - val_loss: 0.0221 - val_acc: 0.9942
Epoch 8/50
796/796 [==============================] - 16s 20ms

In [17]:
model.load_weights('bestmodel.model')

In [18]:
results=model.predict(img_test/255.0)

In [19]:
results=np.argmax(results, axis=1)

In [20]:
dg=pd.DataFrame()
dg['id']=list(test_data.values[0:,0])
dg['label']=results

In [21]:
dg.head()

,id,label
0,0,3
1,1,0
2,2,2
3,3,6
4,4,7


In [ ]:
dg.to_csv("submission.csv", index=False)